In [16]:
# Testing the addition of NCMC moves in saltwap.
# Importing:
from datetime import datetime
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import sys
sys.path.append("../saltswap/")
import saltswap

# Defining constants.
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure = 1*unit.atmospheres
temperature = 300*unit.kelvin
delta_chem = -630*unit.kilojoule_per_mole  # should be around -630 kJ/mol for bulk water with 0.2 M salt.

In [17]:
#Loading a premade water box:
pdb = app.PDBFile('../examples/waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

In [18]:
# Minimizing energy with a temporary integrator.
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context, 1.0, 25)
positions = context.getState(getPositions=True).getPositions(asNumpy=True)
del context, integrator

In [19]:
# Defining the new system integrator that will be used for the standard MD moves:
integrator = openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(pressure, temperature, 25))
# Initialising the constant salt class. Inside, a 'compoundintegrator' is defined to do NCMC
#     NCMC parameters:
nkernals = 1000           # Number of NCMC steps
nverlet = 1            # Number of velocity verlet steps in each NCMC step     
mc_saltswap = saltswap.SaltSwap(system=system,topology=pdb.topology,temperature=temperature,delta_chem=delta_chem,
                                integrator=integrator,pressure=pressure,
                                debug=True,nkernals=nkernals, nverlet_steps=nverlet)

identifyResidues: 501 HOH molecules identified.


In [20]:
# Creating the working the context
platform = openmm.Platform.getPlatformByName('CPU')
context = openmm.Context(system, mc_saltswap.compound_integrator,platform)
context.setPositions(positions)

In [21]:
# Brief thermalisation:
context.setVelocitiesToTemperature(temperature)
integrator.step(10)
positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)

In [22]:
# Cycles of MD and MC salt exchanges:
iterations = 10        # Number of rounds of MD and constant salt moves
nsteps = 100           # Amount of MD steps per iteration. 250000 steps = 500 picoseconds
nattempts = 5        # Number of identity exchanges attempts per iteration 
printfreq = 5

pdbfile = open('output.pdb', 'w')
app.PDBFile.writeHeader(pdb.topology, file=pdbfile)
app.PDBFile.writeModel(pdb.topology, positions, file=pdbfile, modelIndex=0)

startTime = datetime.now()
for i in range(iterations):
    integrator.step(nsteps)
    mc_saltswap.update(context,nattempts=nattempts)
    positions = context.getState(getPositions=True,enforcePeriodicBox=True).getPositions(asNumpy=True)
    app.PDBFile.writeModel(pdb.topology, positions, file=pdbfile, modelIndex=i+1)
    if i%printfreq == 0:
        print "Step %i complete" % i
        print "Identity counts:", mc_saltswap.getIdentityCounts()
        print "Number of NaNs = ",mc_saltswap.nan
        #mc_saltswap.resetStatistics(self)
tm = datetime.now() - startTime

print "\nSimulation time = ",tm.seconds
print "Acceptance probability =",mc_saltswap.getAcceptanceProbability()
print "NaNs = ",mc_saltswap.nan

Step 0 complete
Identity counts: (493, 4, 4)
Number of NaNs =  0
Step 5 complete
Identity counts: (473, 14, 14)
Number of NaNs =  0

Simulation time =  634
Acceptance probability = 0.5
NaNs =  0


With iterations =100, nsteps = 10, nattampts =5:
- No NCMC:
    Simulation time =  7
    Acceptance probability = 0.016
- nkernals = 2, nverlet = 1:
    Simulation time = 24
    Acceptance probability = 0.09
    - repeat:
        Simulation time = 23
        Acceptance probability = 0.04    
- nkernals = 5, nverlet = 1:
   Simulation blows up?
- nkernals = 5, nverlet = 2:   
    Simulation time = 74
    Acceptance probability = 0.63
    - repeat:
        Simulation time = 74
        Acceptance probability = 0.682
- nkernals = 2, nverlet = 2
    Simulation blows up?
- nkernals = 10, nverlet = 2
    Simulation time =  137
    Acceptance probability = 0.07
    - repeat:
        Simulation time =  141
        Acceptance probability = 0.062
- nkernals = 5, nverlet = 5
    Simulation time =  166
    Acceptance probability = 0.06
- nkernals = 100, nverlet = 10, deltachem=-630
    Simulation time = 627
    Acceptance probability = 0.50
- nkernals = 1000, nverlet = 1, deltachem=-630
    Simulation time = 640
    Acceptance probability = 0.4
    
    
With iterations =200, nsteps = 5000, nattampts =5:
 

In [8]:
2833/60

47